# Type4_Consignment

This note is prepared to filter out INCOMPLE TRACES. The below criterias are considered to identify them.  

#### *  Required activities are not included
#### *  Not prper start and End activities
#### * Cases with duraion = 0
#### * ~~ Accumulated values = 0 ~~ (NOT REQUIRED, AS NO INVOICE REQUIRED)

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# IMPORTING

In [2]:
from pm4py.objects.log.importer.xes import importer as xes_importer
log = xes_importer.apply('./data/consignment.xes')

In [3]:
print(
    "Number of trace", len(log),
    "\nNumber of events", sum([len(trace) for trace in log])
)

Number of trace 6866 
Number of events 16070


# FILTERING 

## 1. Filtering by variants


<b> CASE I  </b>
* Create Purchase Order Item  
* ~~ Clear Invoice ~~ (NOT REQUIRED)
* Record Goods Receipt  
    
     
<b> CASE II </b>
* 'Create Purchase Order Item'  
* 'Delete Purchase Order Item'  


In [4]:
from pm4py.algo.filtering.log.variants import variants_filter
variants = variants_filter.get_variants(log)

In [5]:
from pm4py.statistics.traces.log import case_statistics

variants_count = case_statistics.get_variant_statistics(log)
variants_count = sorted(variants_count, key=lambda x: x['count'], reverse=True)

In [6]:
# Filtering out
search=[]
for variant in variants_count :
    try : 
        location_PO=variant['variant'].rfind('Create Purchase Order Item')
        location_goodreceipt=variant['variant'].rfind('Record Goods Receipt')
        location_delete = variant['variant'].rfind('Delete Purchase Order Item')
        # case I : PO, goodreceipt
        if (location_goodreceipt>-1) and (location_PO>-1):  
            search.append(variant['variant'])
            
        # case II : PO, delete 
        # PO exist and the last activity is Delete Purchase Order Item (==Cancellation)
        if (location_PO>-1) and (len(variant['variant']) == location_delete + len('Delete Purchase Order Item')) : 
            search.append(variant['variant'])
    except:
        pass
   

In [7]:
log = variants_filter.apply(log, search)
print("The number of filtered traces : ", len(log))

The number of filtered traces :  6850


## 2. Filtering by Start and End activities 

#### The below are considered as COMPLETE  

Traces starting with 
* Create Purchase Order Item': 6744, 
* Create Purchase Requisition Item': 106}
  
  
    
Traces ending with   
* Record Goods Receipt': 6538,
* Delete Purchase Order Item': 182,


#### The below are REMOVED 

Traces starting with 
* 'Vendor creates invoice'  
  
  
    
Traces ending with   
* 'Change Quantity': 23,  
* 'Change Delivery Indicator': 75,  
* 'Receive Order Confirmation': 4,  
* 'Reactivate Purchase Order Item': 2,  
* 'Cancel Goods Receipt': 25,
* 'Update Order Confirmation': 1

In [8]:
from pm4py.algo.filtering.log.start_activities import start_activities_filter
from pm4py.algo.filtering.log.end_activities import end_activities_filter

log_start = start_activities_filter.get_start_activities(log)
log_end = end_activities_filter.get_end_activities(log)

In [9]:
log_start

{'Create Purchase Order Item': 6744, 'Create Purchase Requisition Item': 106}

In [10]:
log_end

{'Change Quantity': 23,
 'Record Goods Receipt': 6538,
 'Delete Purchase Order Item': 182,
 'Change Delivery Indicator': 75,
 'Receive Order Confirmation': 4,
 'Reactivate Purchase Order Item': 2,
 'Cancel Goods Receipt': 25,
 'Update Order Confirmation': 1}

In [11]:
log = start_activities_filter.apply(log, ['Create Purchase Order Item','Create Purchase Requisition Item' ])
log = end_activities_filter.apply(log, ['Record Goods Receipt', 'Delete Purchase Order Item'])
print("The number of filtered traces : ", len(log))

The number of filtered traces :  6720


## 3. Filtering by case_performance =0  
this means the durations of the traces are 0, which are logically wrong. 

In [32]:
from pm4py.algo.filtering.log.cases import case_filter
log = case_filter.filter_on_case_performance(log, 1, float('inf'))

In [33]:
print("The number of filtered traces : ", len(log))

The number of filtered traces :  6708


## ~~ 4. Filtering by cumulative_neet_worth_(EUR) =0 ~~ (NOT REQUIRED IN THIS TYPE)


# EXPORTING THE FILTERED LOG 

In [34]:
from pm4py.objects.log.exporter.xes import factory as xes_exporter
xes_exporter.export_log(log, "./data/filtered_consignment.xes")

/home/bk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecatedWarning: export_log is deprecated as of 1.3.0 and will be removed in 2.0.0. Use exporter entrypoint instead
  
